In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions

from bs4 import BeautifulSoup
import re

import sys
if 'ipykernel' in sys.modules:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

import pandas as pd
import time

from selenium import webdriver

options = webdriver.ChromeOptions()
#options.add_argument("--headless")
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--no-sandbox")  # sandbox 주요 요소 처리불가-> nosandbox 처리 가능

#headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0'}
#options.add_argument('headers')
driver = webdriver.Chrome(options=options)

In [2]:
url='https://huggingface.co/papers?date=2023-12-13'
driver.get(url)
time.sleep(1)

# 전 날 페이지로 넘기기
# for d in range(2):

all_title=[]
all_news=[]
all_url=[]
all_author=[] 
all_stars=[]
all_days=[]
word=[]
all_images=[]
all_images_url=[]
all_video=[]
all_video_url=[]

titles=driver.find_elements(By.TAG_NAME,'h3')# 제목
stars=driver.find_elements(By.TAG_NAME,'a>div>div') # 좋아요
day=driver.find_elements(By.TAG_NAME,'time')

for i in titles:
    all_title.append(i.text)
all_title=[i for i in all_title if i]  # 빈칸제거

for i in stars:
    all_stars.append(i.text)

for i in day:
    all_days.append(i.text)

print(all_days)
    
word=all_days[0]
word=word.replace('\n','')
all_days2=[]
all_days2.append(word)

count=0
for n in all_title:
    count+=1
    print(n)
    driver.find_element(By.PARTIAL_LINK_TEXT,n).click() #페이지 들어가기
    time.sleep(1)
    
    nourl=driver.current_url#url 저장
    all_url.append(nourl)
    time.sleep(1)
    
    news=driver.find_element(By.TAG_NAME,'p') # 뉴스 본문 저장
    all_news.append(news.text)


    author=driver.find_element(By.CLASS_NAME,'whitespace-nowrap.underline') #저자 
    all_author.append(author.text)


    # if driver.find_elements(By.TAG_NAME,'img'): #이미지 paper 에는 이미지 없음
    #     all_images.append('yes')
    #     image=driver.find_element(By.XPATH,'/html/body/div/main/div[2]/section[1]/div[3]/div[2]/p[4]/a/img')
    #     img_srtc=image.get_attribute('src')
    #     all_images_url.append(img_srtc)
    # else:
    #     all_images.append('yes')
        

    if driver.find_elements(By.TAG_NAME,'video'): # 동영상
        all_video.append('yes')
        image=driver.find_element(By.XPATH,'/html/body/div/main/div/section[1]/div/video')
        img_srtc=image.get_attribute('src')
        all_video_url.append(img_srtc)
    else:
        all_video.append('no')
        all_video_url.append('-')

    driver.back()
    time.sleep(1)
    
    if count>3:
        break


    # # 잠깐 쉬고 전 페이지 가기
    # time.sleep(3)
    # driver.find_element(By.CLASS_NAME,'btn.gap-2').click() # 사이트 전 날

all_days2=all_days2*len(all_title) # 날짜 개수 제목개수만큼 늘리기


sumnews=[]
for t,n,u,a,s,d,v,vr in zip(all_title,all_news,all_url,all_author,all_stars,all_days2,all_video,all_video_url):
    singles={'title':t,'news':n,'url':u,'author':a,'like':s,'day':d,'video':v,'video_url':vr}
    sumnews.append(singles)

import pandas as pd
news_df=pd.DataFrame(sumnews)
print(news_df)

import os
if not os.path.exists('output.csv'):
    news_df.to_csv('output.csv', index=False, mode='w', encoding='utf-8-sig')
else:
    news_df.to_csv('output.csv', index=False, mode='a', encoding='utf-8-sig', header=False)

['DEC\n13']
FreeInit: Bridging Initialization Gap in Video Diffusion Models
FreeControl: Training-Free Spatial Control of Any Text-to-Image Diffusion Model with Any Condition
VILA: On Pre-training for Visual Language Models
DiffMorpher: Unleashing the Capability of Diffusion Models for Image Morphing
                                               title  \
0  FreeInit: Bridging Initialization Gap in Video...   
1  FreeControl: Training-Free Spatial Control of ...   
2   VILA: On Pre-training for Visual Language Models   
3  DiffMorpher: Unleashing the Capability of Diff...   

                                                news  \
0  Though diffusion-based video generation has wi...   
1  Recent approaches such as ControlNet offer use...   
2  Visual language models (VLMs) rapidly progress...   
3  Diffusion models have achieved remarkable imag...   

                                        url        author like    day video  \
0  https://huggingface.co/papers/2312.07537   Tianxing Wu

In [18]:

for i in news_df['video_url']:
    if i!='-':
        print(i)

https://cdn-uploads.huggingface.co/production/uploads/60f1abe7544c2adfd699860c/pPBftW9u_SvGH1HLzFDu-.mp4
https://cdn-uploads.huggingface.co/production/uploads/60f1abe7544c2adfd699860c/BEB9BbR70W_YtxAyw6TvR.qt


In [3]:
news_df

,title,news,url,author,like,day,video,video_url
0,FreeInit: Bridging Initialization Gap in Video...,Though diffusion-based video generation has wi...,https://huggingface.co/papers/2312.07537,Tianxing Wu,21,DEC13,no,-
1,FreeControl: Training-Free Spatial Control of ...,Recent approaches such as ControlNet offer use...,https://huggingface.co/papers/2312.07536,Sicheng Mo,12,DEC13,no,-
2,VILA: On Pre-training for Visual Language Models,Visual language models (VLMs) rapidly progress...,https://huggingface.co/papers/2312.07533,Ji Lin,12,DEC13,no,-
3,DiffMorpher: Unleashing the Capability of Diff...,Diffusion models have achieved remarkable imag...,https://huggingface.co/papers/2312.07409,Kaiwen Zhang,11,DEC13,yes,https://cdn-uploads.huggingface.co/production/...


In [3]:

url='https://huggingface.co/papers/2312.07409'
driver.get(url)

all_video=[]
all_video_url=[]


if driver.find_elements(By.TAG_NAME,'video'): # 동영상
    all_video.append('yes')
    image=driver.find_element(By.XPATH,'/html/body/div/main/div/section[1]/div/video')
    img_srtc=image.get_attribute('src')
    all_video_url.append(img_srtc)
    print(all_video_url)
else:
    all_video.append('no')

['https://cdn-uploads.huggingface.co/production/uploads/60f1abe7544c2adfd699860c/pPBftW9u_SvGH1HLzFDu-.mp4']
